In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer

# Cargar data_descartada desde el archivo CSV
data_descartada = pd.read_csv('data_descartada.csv')

# Variables independientes (X) y variable dependiente (y)
X = data_descartada.drop(columns=['NRO TOTAL MICROCREDITO', 'NRO DEPOSITOS', 'NRO GIROS ENVIADOS', 'NRO GIROS RECIBIDOS'])
y = data_descartada['NRO TOTAL MICROCREDITO']

# Imputar los valores faltantes
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Estandarizar los datos
scaler = StandardScaler()
X_std = scaler.fit_transform(X_imputed)

# Modelo de Gradient Boosting
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)

# Validación cruzada y predicciones
cv_folds = 3  # Cambia el número de pliegues aquí
y_pred_cv = cross_val_predict(gb_model, X_std, y, cv=cv_folds, n_jobs=-1)

# Funciones para calcular y mostrar métricas
def calculate_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    metrics = {'MSE': mse, 'MAE': mae, 'R2': r2}
    return metrics

def display_regression_metrics(metrics_dict):
    print("Métricas de Regresión:")
    for metric, value in metrics_dict.items():
        print(f"{metric}: {value:.4f}")

# Calcular y mostrar métricas de regresión
metrics_cv = calculate_regression_metrics(y, y_pred_cv)
display_regression_metrics(metrics_cv)


Métricas de Regresión:
MSE: 0.0000
MAE: 0.0000
R2: 0.4827


Comparando los resultados obtenidos en el modelo de Gradient Boosting utilizando validación cruzada con los resultados anteriores, se pueden observar algunos cambios en el desempeño del modelo. A continuación, describiré los cambios y proporcionaré posibles razones para estos cambios:

1. Resultados Anteriores (Entrenamiento y Prueba):

- Mean Squared Error (MSE): 971.97
- R-squared (R2): 0.9989

2. Resultados con Validación Cruzada:

- Mean Squared Error (MSE): 0.0000
- Mean Absolute Error (MAE): 0.0000
- R-squared (R2): 0.4827


Cambios en el Desempeño:

- MSE y MAE:
    - Los resultados anteriores mostraban un MSE de 971.97 en el conjunto de prueba, lo que indicaba un nivel razonable de ajuste y error moderado.
    - En los resultados de validación cruzada, el MSE y MAE son muy cercanos a cero, lo que sugiere un ajuste perfecto en la muestra utilizada.
- R2:
    - El R2 anterior indicaba que aproximadamente el 99.89% de la variabilidad de los datos se explicaba por el modelo.
    - El R2 en los resultados de validación cruzada es más bajo, alrededor del 48.27%, lo que sugiere que el modelo explica menos variabilidad en los datos.

Razones Posibles:

- Sobreajuste: Los resultados de validación cruzada muestran un ajuste perfecto en la muestra utilizada. Esto podría deberse a un posible sobreajuste del modelo a los datos de entrenamiento. Es decir, el modelo puede haber aprendido los datos de entrenamiento tan específicamente que no generaliza bien a nuevos datos.

- Tamaño de la Muestra: La muestra utilizada en la validación cruzada podría ser pequeña en relación con la complejidad del modelo. Un tamaño de muestra pequeño podría llevar a un ajuste excesivo y a resultados poco realistas.

- Representatividad de los Datos: Los datos utilizados en la validación cruzada podrían no ser completamente representativos de la población general. Si los datos no son una muestra adecuada, los resultados de validación cruzada pueden ser engañosos.

- Selección de Características: La selección de características puede tener un impacto significativo en el desempeño del modelo. Si se utilizan características irrelevantes o se omiten características importantes, los resultados pueden variar drásticamente.

En general, los resultados de validación cruzada muestran un desempeño extremadamente optimista y poco realista del modelo. Es importante considerar estos resultados en el contexto de las razones mencionadas anteriormente. Para obtener una evaluación más precisa del rendimiento del modelo, se recomienda utilizar un conjunto de datos más grande y aplicar técnicas de validación cruzada más robustas, como la estratificación y el aumento del número de pliegues, para evitar conclusiones incorrectas basadas en un ajuste perfecto en una muestra pequeña.

### Utilizaremos la técnica de regresión Ridge para aplicar regularización y asi superar el sobreajuste que tenemos como problema

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Cargar el archivo CSV
DATA_PATH = 'Inclusion_Financiera.csv'
data = pd.read_csv(DATA_PATH)

# Cargar data_descartada desde el archivo CSV
data_descartada = pd.read_csv('data_descartada.csv')

# Variables independientes (X) y variable dependiente (y)
X = data_descartada.drop(columns=['NRO TOTAL MICROCREDITO', 'NRO DEPOSITOS', 'NRO GIROS ENVIADOS', 'NRO GIROS RECIBIDOS'])
y = data_descartada['NRO TOTAL MICROCREDITO']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un imputador SimpleImputer
imputer = SimpleImputer(strategy='mean')

# Imputar los valores faltantes en los datos de entrenamiento y prueba
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Estandarizar los datos
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train_imputed)
X_test_std = scaler.transform(X_test_imputed)

# Entrenar el modelo con regularización Ridge
ridge_model = RidgeCV(alphas=[0.01, 0.1, 1.0, 10.0], cv=5)
ridge_model.fit(X_train_std, y_train)

# Realizar predicciones en los datos de prueba
y_pred = ridge_model.predict(X_test_std)

# Calcular y mostrar métricas de regresión
metrics = {
    'MSE': mean_squared_error(y_test, y_pred),
    'MAE': mean_absolute_error(y_test, y_pred),
    'R2': r2_score(y_test, y_pred)
}

# Mostrar métricas
print("Métricas de Regresión con Regularización Ridge:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/extmath.py:193: RuntimeWar

Métricas de Regresión con Regularización Ridge:
MSE: 0.0000
MAE: 0.0000
R2: 1.0000


Los resultados obtenidos indican que el modelo de regresión Ridge con regularización ha logrado ajustarse muy bien a los datos de prueba. Las métricas de regresión (MSE, MAE, R2) son excelentes y en particular el coeficiente de determinación (R2) es 1.0000, lo que significa que el modelo está explicando la variabilidad de los datos de manera perfecta.

Este resultado es bastante sorprendente y poco común. En la mayoría de los casos, un R2 de 1.0000 indica un posible problema, como sobreajuste o errores en los datos. Es importante considerar que este resultado puede ser debido a la naturaleza particular de los datos, la cantidad de características y la forma en que se han procesado y utilizado.